In [ ]:
!pip install transformers

In [ ]:
#!pip install torchaudio

In [ ]:
!pip install hydra-core omegaconf

In [ ]:
!pip install bitarray

In [ ]:
!pip uninstall fairseq
!pip install fairseq

In [ ]:
!pip install fairseq
import os
# os.makedirs('/content/drive/MyDrive/535/WAV2VEC2', exist_ok=True)

%cd /content/drive/MyDrive/535/WAV2VEC2

!fairseq-hub install 'facebook/wav2vec2-large-960h-lv60-self'


In [ ]:
!mkdir -p /content/drive/MyDrive/535/WAV2VEC2_XLSR53
!wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec2-large-xlsr-53.zip -P /content/drive/MyDrive/535/WAV2VEC2_XLSR53
!unzip /content/drive/MyDrive/535/WAV2VEC2_XLSR53/wav2vec2-large-xlsr-53.zip -d /content/drive/MyDrive/535/WAV2VEC2_XLSR53


-----------------------------------------------------

In [ ]:
import os
import numpy as np
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import json

# Load the pre-trained wave2vec model
model_name = 'facebook/wav2vec2-large-960h-lv60-self'
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Load the metadata
metadata_path = '/content/drive/MyDrive/535/IEMOCAP_full_release/meta_data/meta_data.json'

with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Define a list to store the features and labels
features = []
labels = []

audio_dir = '/content/drive/MyDrive/535/IEMOCAP_full_release/'
counter = 0
# Loop through each item in the metadata file
for item in metadata['meta_data']:
    if counter == 3000:
        break
    counter+=1
    print(counter)
    # Load the audio file and extract the relevant segment
    audio_file_path = os.path.join(audio_dir, item['path'])
    emotion_label = item['label']

    input_audio, _ = sf.read(audio_file_path, dtype='float32')

    # Convert the audio segment to a PyTorch tensor of input values
    input_values = tokenizer(input_audio, return_tensors='pt').input_values

    # Pass the input values through the Wav2Vec2 model to obtain the features
    with torch.no_grad():
        output = model(input_values)
        features_tensor = output.logits.squeeze().cpu().numpy()

    # Store the features and labels in the lists
    features.append(features_tensor)
    labels.append(metadata['labels'][emotion_label])
    
# Convert the lists to numpy arrays
features = np.array(features)
labels = np.array(labels)


In [ ]:
import pandas as pd
# Extracted feature vectors are stacked into a single numpy array called `features`.

features_df = pd.DataFrame(features,columns=['features'])
labels_df = pd.DataFrame(labels, columns=['label'])

# Concatenate the features, labels, and file names along the columns.
all_data_df = pd.concat([features_df, labels_df ], axis=1)

# Save the features as a CSV file.
all_data_df.to_csv('/content/drive/MyDrive/535/WAV2VEC2/feaures.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

# Flatten feature vectors using numpy.ravel() method.
features_flat = np.array([f.ravel() for f in features])

# Convert flattened feature vectors to a pandas DataFrame.
features_df = pd.DataFrame(features_flat, columns=['features'])

# Create a DataFrame for labels.
labels_df = pd.DataFrame(labels, columns=['label'])

# Concatenate the features, labels, and file names along the columns.
all_data_df = pd.concat([features_df, labels_df], axis=1)

# Save the features as a CSV file.
all_data_df.to_csv('/content/drive/MyDrive/535/WAV2VEC2/features1.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
import os

# Flatten feature vectors using numpy.ravel() method.
features_flat = np.array([f.ravel() for f in features])

# Split the features into chunks of maximum 10,000 records.
chunk_size = 10000
num_chunks = (len(features_flat) - 1) // chunk_size + 1
feature_chunks = np.array_split(features_flat, num_chunks)

# Create a DataFrame for labels.
labels_df = pd.DataFrame(labels, columns=['label'])

# Write each feature chunk to a separate file.
for i, chunk in enumerate(feature_chunks):
    chunk_df = pd.DataFrame(chunk, columns=['features'])
    all_data_df = pd.concat([chunk_df, labels_df], axis=1)
    file_name = f'/content/drive/MyDrive/535/WAV2VEC2/features_{i}.csv'
    all_data_df.to_csv(file_name, index=False)

# Merge all the feature files into a single file.
file_names = [f'/content/drive/MyDrive/535/WAV2VEC2/features_{i}.csv' for i in range(num_chunks)]
all_data_df = pd.concat([pd.read_csv(f) for f in file_names], ignore_index=True)

# Save the features as a CSV file.
all_data_df.to_csv('/content/drive/MyDrive/535/WAV2VEC2/features_formatted.csv', index=False)

# Remove the temporary files.
for file_name in file_names:
    os.remove(file_name)


<ipython-input-111-02219c5c121c>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features_flat = np.array([f.ravel() for f in features])


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Load the features and labels from the CSV file
all_data_df = pd.read_csv('/content/drive/MyDrive/535/WAV2VEC2/features_formatted.csv')

all_data_df['label'][0] = 0

# Remove any rows with missing values
all_data_df = all_data_df.dropna()

# Convert the features column from string to float
all_data_df['features'] = all_data_df['features'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
all_data_df = all_data_df[all_data_df['features'].apply(lambda x: x.shape == (3,))]
# for i, row in all_data_df.iterrows():
#     print(f"Shape of feature array {i}: {row['features'].shape}")
    

# Split the data into training and testing sets
X = np.stack(all_data_df['features'].values)
y = all_data_df['label'].values.astype(int) # cast the labels to integer type

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an SVC model
svc_model = SVC(kernel='linear', C=1, gamma='scale')
svc_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svc_model.predict(X_test)

# Print the classification report.
print(classification_report(y_test, y_pred, zero_division=1))

	        precision 	 recall 	f1-score 	support
	    0 	     0.69	   0.87 	    0.79 	     20
	    1 	     0.75	   0.65 	    0.71 	     23
	    2 	     0.43	   0.42 	    0.44 	     11
	    3 	     0.56	   0.61 	    0.59 	     18
	    4 	     0.55	   0.61 	    0.59 	     18
	    5 	     0.56	   0.61 	    0.59 	     18
     accuracy  	         	       	    	    0.695	     54
    macro avg 	     0.58	   0.60 	    0.60 	     34
 weighted avg 	     0.64	   0.66 	    0.66 	     45
